# Hello Julia

とりあえずJuliaを触ってみる。触ってみるだけ。

In [1]:
s = "Hello Julia!"
println(s);

Hello Julia!


# 基本構文

## 変数

In [2]:
x = 1
y = 2.0
z = x + y

3.0

こういう書き方もできる。変数の前に数値を置くと暗黙的に乗算と解釈される。

In [3]:
2x + 1

3

## 定数

In [4]:
const a = 1

1

In [5]:
a = 2

2

In [6]:
pi

π = 3.1415926535897...

In [7]:
pi = 3

LoadError: cannot assign a value to variable MathConstants.pi from module Main

In [8]:
pi

π = 3.1415926535897...

## 文字列の補間

In [9]:
"1 + 2 = $(1 + 2)"

"1 + 2 = 3"

# 制御構文

## 条件評価

In [10]:
x = 1
y = 2

if x < y
    println("x < y")
elseif x > y
    println("x > y")
else
    println("x = y")
end

x < y


三項演算子

In [11]:
x < y ? "x<y" : "x>=y"

"x<y"

## 短絡評価

複数の条件を評価する。
最初に書かれた条件の結果によって二つ目の評価を飛ばすことがある。

a && b の場合に、aの結果がFalseならこの結果は必ずFalseになるのでbの評価をしない。

In [12]:
n = 1
n < 0 && error("n must be positive")

false

## ループ

In [13]:
i = 1
while i <= 3
    println("i = $(i)")
    i += 1
end

i = 1
i = 2
i = 3


In [14]:
for j = 1:3
    println("j = $(j)")
end

j = 1
j = 2
j = 3


forやwhileなどのブロック内で使われる変数はブロック内のローカル変数となる。
なので、ブロック外で使用するとエラーが出る。

In [15]:
j

LoadError: UndefVarError: j not defined

## 関数

関数は基本は`function`~`end`で囲む。returnを明示しない場合、最後に評価された値が戻り値となる。

型は指定してもしなくてもOK。

In [16]:
function add(x::Int, y::Int)::Int
    return x + y
end

add (generic function with 1 method)

In [17]:
add(1, 3)

4

1行で関数を書ける場合

In [18]:
add(x::Int, y::Int)::Float64 = x + y

add (generic function with 1 method)

In [19]:
add(3, 4)

7.0

戻り値が複数の場合

In [20]:
function add_diff(x, y)
    return (x+y, x-y)
end

# add_diff(x, y) = (x+y, x-y);

In [21]:
add_diff(4, 3)

(7, 1)

可変長引数

In [22]:
function f(x...)
    sum = 0
    for i = 1:length(x)
        sum += x[i]
    end
    return sum
end

f (generic function with 1 method)

In [23]:
f(1)

1

In [24]:
f(1,2,3,4,5)

15

オプショナル引数

引数名を指定できないけど、デフォルト引数を指定できる。

In [25]:
function f_op(x, y=1)
    return x + y
end

f_op (generic function with 2 methods)

In [26]:
f_op(1)

2

In [27]:
f_op(1, 0)

1

In [28]:
f_op(1, y=0)

LoadError: MethodError: no method matching f_op(::Int64; y=0)
[0mClosest candidates are:
[0m  f_op(::Any) at In[25]:1[91m got unsupported keyword argument "y"[39m
[0m  f_op(::Any, [91m::Any[39m) at In[25]:1[91m got unsupported keyword argument "y"[39m

キーワード引数

引数名を指定できる。

In [29]:
function f_kw(x; y=1)
    return x + y
end

f_kw (generic function with 1 method)

In [30]:
f_kw(1)

2

In [31]:
f_kw(1, y=0)

1

In [32]:
f_kw(1, 0)

LoadError: MethodError: no method matching f_kw(::Int64, ::Int64)
[0mClosest candidates are:
[0m  f_kw(::Any; y) at In[29]:1

# 型

## 複合型

structで宣言する。複数の名前付きフィールドをまとめる。

In [33]:
struct Point
    x::Int
    y::Int
end

In [34]:
function distance(p::Point)
    sqrt(p.x^2 + p.y^2)
end

distance (generic function with 1 method)

In [35]:
p = Point(2, 3)
distance(p)

3.605551275463989

In [36]:
p.x

2

In [37]:
Point(x) = Point(x, 0)

Point

In [38]:
p = Point(1)
p

Point(1, 0)

In [39]:
Point(1, 5)

Point(1, 5)

structで宣言したフィールドはオブジェクトの作成の後からは変更できなくなる

In [40]:
p.x = 2

LoadError: setfield! immutable struct of type Point cannot be changed

mutableなstruct宣言をすると、後からフィールドの値を変更できる

## パラメトリック型

In [41]:
mutable struct Point_p{T}
    x::T
    y::T
end;

In [42]:
p = Point_p(2, 3) # TはIntと解釈される

Point_p{Int64}(2, 3)

In [43]:
p = Point_p(2.1, 3.) # TはFloatと解釈される

Point_p{Float64}(2.1, 3.0)

In [44]:
function distance(p::Point_p{T}) where T
    sqrt(p.x^2 + p.y^2)
end

distance (generic function with 2 methods)

In [45]:
distance(p)

3.661966684720111

型によって動作を変える場合は、多重ディスパッチを利用する

In [46]:
function distance(p::Point_p{Int})
    println("int")
    sqrt(p.x^2 + p.y^2)
end;

function distance(p::Point_p{Float64})
    println("float64")
    sqrt(p.x^2 + p.y^2)
end;

In [47]:
p = Point_p(2, 3)
distance(p)

int


3.605551275463989

In [48]:
p = Point_p(2.5, 3.5)
distance(p)

float64


4.301162633521313

# コレクション

## タプル

タプルはimmutableなので値を変更できない。

可変長引数はタプルになる。\

In [49]:
f(x...) = x;
y = f(3, 5, 7)

println(y)
println(typeof(y))

(3, 5, 7)
Tuple{Int64, Int64, Int64}


## リスト

1次元のArrayをリストと呼ぶ(Pythonと一緒)

In [50]:
l = [1, 2];
println(l)

push!(l, 5);
println(l)

z = pop!(l);
println(z)

insert!(l, 2, 9);
println(l)

[1, 2]
[1, 2, 5]
5
[1, 9, 2]


## 辞書

In [51]:
d = Dict{String, Int}()
d["a"] = 1
d["b"] = 2
d

Dict{String, Int64} with 2 entries:
  "b" => 2
  "a" => 1

In [52]:
for (key, value) in d
    println(key)
    println(value)
end

b
2
a
1


In [53]:
d = Dict{String, Int}()
d["a"] = 1
d["b"] = 2
d["c"] = 5

next = iterate(d)
while next != nothing
    (x, state) = next
    println(x)
    next = iterate(d, state)
end

"c" => 5
"b" => 2
"a" => 1


In [54]:
d = Dict{String, Int}()
d["a"] = 1
d["b"] = 2
d["c"] = 5

next = iterate(d)

("c" => 5, 5)

In [55]:
l = [1, 2, 3, 5, 7, 9];
next = iterate(l)
while next != nothing
    (x, state) = next
    println(x)
    next = iterate(l, state)
end

1
2
3
5
7
9


In [56]:
l = [1, 2, 3, 5, 7, 9];
next = iterate(l)

(1, 2)

# 多次元配列

In [57]:
# 値が初期化されない3x2の多次元配列
Array{Float32}(undef, 3, 2)

3×2 Matrix{Float32}:
 0.0      0.0
 0.0      1.0f-45
 1.0f-45  0.0

In [58]:
ones(Float32, 3, 2)

3×2 Matrix{Float32}:
 1.0  1.0
 1.0  1.0
 1.0  1.0

In [59]:
rand(Float32, 2, 3)

2×3 Matrix{Float32}:
 0.744162  0.722148  0.506676
 0.823919  0.378753  0.667184

In [60]:
rand(Float32, 2, 3, 2)

2×3×2 Array{Float32, 3}:
[:, :, 1] =
 0.185149  0.971972  0.46967
 0.674011  0.55657   0.0756104

[:, :, 2] =
 0.264081  0.0126067  0.562862
 0.403037  0.167569   0.465194

In [61]:
A = rand(Float32, 3, 2);
size(A)

(3, 2)

In [62]:
strides(A)

(1, 3)

In [63]:
A

3×2 Matrix{Float32}:
 0.948274  0.403252
 0.259121  0.763557
 0.46172   0.716286

In [64]:
A[1,2]

0.403252f0

In [65]:
A[:,1]

3-element Vector{Float32}:
 0.9482744
 0.25912058
 0.46171963

In [66]:
A[3, 1] = 1.0
A

3×2 Matrix{Float32}:
 0.948274  0.403252
 0.259121  0.763557
 1.0       0.716286

ドット演算子は要素毎の演算となる（matlabみたいな）

In [67]:
A = rand(Float32, 3, 3)

3×3 Matrix{Float32}:
 0.269472  0.796187  0.77052
 0.91752   0.4603    0.334679
 0.977406  0.62454   0.496514

In [68]:
B = ones(Float32, 3, 1)

3×1 Matrix{Float32}:
 1.0
 1.0
 1.0

In [69]:
A + B

LoadError: DimensionMismatch("dimensions must match: a has dims (Base.OneTo(3), Base.OneTo(3)), b has dims (Base.OneTo(3), Base.OneTo(1)), mismatch at 2")

In [70]:
A .+ B

3×3 Matrix{Float32}:
 1.26947  1.79619  1.77052
 1.91752  1.4603   1.33468
 1.97741  1.62454  1.49651

スカラを処理する関数を定義して、行列演算にする場合もドット演算子が使える

In [71]:
sigmoid(x::Float64) = 1.0 / (1.0 + exp(-x));

In [72]:
sigmoid(0.0)

0.5

In [85]:
A = rand(Float64, 3, 3)

3×3 Matrix{Float64}:
 0.48313    0.91538   0.380251
 0.961745   0.943753  0.598667
 0.0416099  0.151734  0.769571

In [86]:
sigmoid.(A)

3×3 Matrix{Float64}:
 0.618487  0.7141    0.593934
 0.723471  0.719857  0.645351
 0.510401  0.537861  0.683428

mapを使っても同じことが書ける

In [87]:
map(sigmoid, A)

3×3 Matrix{Float64}:
 0.618487  0.7141    0.593934
 0.723471  0.719857  0.645351
 0.510401  0.537861  0.683428

# モジュール

既存モジュールから名前を取り込む（既存のモジュールを使う）には `using` を使う

```
using Statistics: mean
```

のようにして、特定の名前だけを取り込むこともできる。

In [75]:
using Statistics

In [76]:
mean([1,2,3])

2.0

In [77]:
using Statistics: Statistics

In [78]:
Statistics.mean([1,2,3,4])

2.5

「モジュール」を取り込んで、モジュールを通してその中の名前を参照できる。こうすると、意図しない名前の衝突が避けられる

モジュールの定義には `module` を使う。慣習として、モジュール内ではインデントしない（なんで？？？）

In [79]:
module Greeting
hello(name) = println("Hello, $(name).")
end

Main.Greeting

In [80]:
Greeting.hello("julia")

Hello, julia.


In [81]:
module Greeting2
hello(name) = println("Hello, $(name).")
function hellohello(name)
    println("Hello, Hello, $(name).")
end
end

Main.Greeting2

In [82]:
Greeting2.hellohello("jul")

Hello, Hello, jul.


using で定義したmoduleの名前を取り込む。　LOAD_PATHに指定されているpathから探すので、相対パスを指定する

In [83]:
using .Greeting2

In [84]:
hello("julia")

LoadError: UndefVarError: hello not defined

??名前が取り込まれてないぞ？？？？？

`include`を使うと、ファイルを取り込むことができる。

取り込んだファイルが`include`を書いたところで展開されるようになる。

In [88]:
include("../src/greeting.jl")

hellohello (generic function with 1 method)

In [89]:
hellohello("julia")

Hello, julia.


# メタプログラミング

Pythonでいうところのデコレータ

In [90]:
ex = 1;
:(2x + $(ex))

:(2x + 1)

In [91]:
ex = :(3y + 1);
:(2x + $(ex))

:(2x + (3y + 1))

In [92]:
x = 10
y = 1
eval(:(2x + $(ex)))

24

ビットフラグを持つ3個の定数

In [94]:
for (i, name) in enumerate([:A, :B, :C])
    @eval const $(Symbol(:FLAG_, name)) = $(UInt16(1) << (i - 1))
end

In [95]:
FLAG_A, FLAG_B, FLAG_C

(0x0001, 0x0002, 0x0004)

In [96]:
FLAG_A

0x0001

## マクロ機能

In [98]:
macro plus1(ex)
    :($(ex) + 1)
end

@macroexpand @plus1 2x

:(2 * Main.x + 1)

マクロで何もせずに変数を指定すると、グローバル変数として振る舞う。

In [100]:
macro plus1e(ex)
    :($(esc(ex)) + 1)
end

@macroexpand @plus1e 2x

:(2x + 1)

esc関数を使って識別子がグルーバル変数に変換されないようにする。

こうしないと、関数内やループの中でマクロを呼び出したときに、意図せずグローバルな変数を参照するようになってしまう。

### コードの実行時間を計測するマクロ

In [101]:
macro time_ns(ex)
    quote
        t1 = time_ns()
        val = $(esc(ex))
        t2 = time_ns()
        val, Int(t2 - t1)
    end
end

@time_ns (macro with 1 method)

In [104]:
sum(randn(1000))

-52.09567940534663

In [105]:
@macroexpand @time_ns sum(randn(1000))

quote
    #= In[101]:3 =#
    var"#31#t1" = Main.time_ns()
    #= In[101]:4 =#
    var"#32#val" = sum(randn(1000))
    #= In[101]:5 =#
    var"#33#t2" = Main.time_ns()
    #= In[101]:6 =#
    (var"#32#val", Main.Int(var"#33#t2" - var"#31#t1"))
end

macro内で使っている変数が切り替わっている（衛生的マクロ）ことがわかる

# 外部プログラムの呼び出し

`` で囲うと、コマンドCmd型オブジェクトになる。

In [106]:
`ls`

`ls`

In [107]:
run(`ls`)

00_hello_julia.ipynb


Process(`ls`, ProcessExited(0))

標準出力を変数に取り込む場合は、readを使う

In [108]:
ret = read(`ls`, String)
println(ret)

00_hello_julia.ipynb



## パイプ処理

`|`は文字として解釈されるので、パイプで連続処理をしたい場合には、`pipeline`を使う

In [109]:
ret = pipeline(`cat ../README.md`, `wc -l`)
run(ret)

11


Base.ProcessChain(Base.Process[Process(`cat ../README.md`, ProcessExited(0)), Process(`wc -l`, ProcessExited(0))], Base.DevNull(), Base.DevNull(), Base.DevNull())

## 補間

In [110]:
cmd = "echo"
run(`$(cmd) hello`)

hello


Process(`echo hello`, ProcessExited(0))

In [111]:
msg = "hello"
run(`echo $(msg)`)

hello


Process(`echo hello`, ProcessExited(0))

In [112]:
`touch $(["foo", "bar"])`

`touch foo bar`

補間の入力をリストやタプルにすると、複数の引数を入力することになる。